# Transform afids-macaca points between templates via the computed warps

**Authors:** Nikoloz Sirmpilatze (NSirmpilatze@dpz.eu)

**Last updated:** 28 Feb 2020

## Import libraries

In [1]:
import os
import numpy as np
import pandas as pd

## Define paths 

In [29]:
os.chdir(os.getcwd())

# templates (Replace with your local template paths)
templates = '/home/nsirmpilatze/home@dpz/RheMAP_data/templates/'
mni = templates + 'MNI/macaque_25_model-MNI.nii.gz'
d99 = templates + 'D99/D99_template.nii'

# Warps (Replace with your local RheMAP warp paths)
warps = '/home/nsirmpilatze/home@dpz/RheMAP_data/warps/'
d99_to_mni_linear = warps + 'linear/D99_to_MNI_affine_0GenericAffine.mat'
d99_to_mni_composite = warps + 'final/D99_to_MNI_CompositeWarp.nii.gz'
mni_to_d99_composite = warps + 'final/MNI_to_D99_CompositeWarp.nii.gz'

# fiducials (example fiducial stored within the repository)
fiducials = './fiducials/Raters_Session01/'
mni_afids = fiducials + 'Fid32_macaqueMNI_T1_GG_01.fcsv'
d99_afids = fiducials + 'Fid32_d99_T1_GG_01.fcsv'
out_dir = './fiducials/transformed/'

## Define function for transforming fcsv files

In [23]:
def transform_fcsv(input_fcsv, output_fcsv, transform, invert=0):
    '''Applies ANTs transform to 3D slicer fcsv file in 3 steps:
        
    1. Slicer fcsv (RAS) is converted to ANTs-compatible (LPS) csv
    2. The transform is applied to the csv via antsApplyTransformsToPoints
    3. The transformed csv is converted back to Slicer fcsv format
    Function is based on afids-tools legacy scripts github.com/afids/afids-tools

    Parameters
    ----------
    input fcsv: path to input Slicer fcsv file
    ouput fcsv: path to output Slicer fcsv file
    transform: path to ANTs transform file (either linear .mat or .nii.gz warp)
    invert: if 1, linear .mat is inverted
    '''

    # get output directory
    output_dir = os.path.dirname(output_fcsv)
    # temporary csv files are also saved in the same output directory
    orig_csv = os.path.join(output_dir, 'tmp_orig.csv')
    transformed_csv = os.path.join(output_dir, 'tmp_transformed.csv')

    # convert Slicer RAS oriented FCSV (default)
    # to Ants LPS oriented format (expected orientation)
    # use with CAUTION: orientation flips here
    df = pd.read_csv(input_fcsv, skiprows=2)
    coords = df[['x', 'y', 'z']]
    coords.loc[:, 't'] = np.zeros(len(coords)) # add a 4th dimension of zeros 
    coords['x'] = -1 * coords['x'] # flip orientation in x
    coords['y'] = -1 * coords['y'] # flip orientation in y
    coords.to_csv(orig_csv, index=False, float_format='%.3f')

    # apply transforms to original csv and get transformed csv
    !antsApplyTransformsToPoints -d 3 -i $orig_csv -o $transformed_csv -t [$transform, $invert]

    new_coords = pd.read_csv(transformed_csv)
    # flip x and y signs, to convert back from ANTs LPS to slicer RAS space
    df['x'] = -1 * new_coords['x'].values.round(3)
    df['y'] = -1 * new_coords['y'].values.round(3)
    df['z'] = new_coords['z'].values.round(3)

    # read lines from input_fcsv
    with open(input_fcsv, 'r') as file:
        lines = file.readlines()
    # replace data lines (leave header unchanged)
    for i in df.index:
        row_entries = [str(cell) for cell in df.iloc[i, :].values]
        lines[3 + i] = ','.join(row_entries) + '\n'
    # write lines to outout_fcsv
    with open(output_fcsv, 'w') as file:
        file.writelines(lines)

## Apply transforms to points

**Important note**: ANTs transforms on points work in the opposite direction (as compared to image transforms).

For example, let's imagine we have to go from space A to space B either using a linear transform (.mat) or a non-linear warp(.nii.gz)
* Linear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B.mat`
* Linear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B -t [A-to-B.mat, 1]` (inverting the forward transform)
* Nonlinear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B_1Warp.nii.gz` (for RheMAP we would use the `A-to-B_CompositeWarp.nii.gz`)
* Nonlinear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B A-to-B_1InverseWarp.nii.gz` (for RheMAP we would use the `B-to-A_CompositeWarp.nii.gz`)

In [31]:
# Linear transform from D99 to MNImacaque
linear_out = out_dir + 'Fid32_d99_GG_T1_01_in-MNImacaque_Linear.fcsv'
transform_fcsv(d99_afids, linear_out, d99_to_mni_linear, 1)

# Composite warp (affine + SyN) from D99 to MNImacaque
composite_out = out_dir + 'Fid32_d99_GG_T1_01_in-MNImacaque_Composite.fcsv'
transform_fcsv(d99_afids, composite_out, mni_to_d99_composite, 0)